In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from joblib import dump
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import string

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
data = pd.read_csv('Twitter_Data.csv')

In [10]:
data.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         27481 non-null  object
 1   text           27480 non-null  object
 2   selected_text  27480 non-null  object
 3   sentiment      27481 non-null  object
dtypes: object(4)
memory usage: 858.9+ KB


In [12]:
print(data.isnull().sum())


textID           0
text             1
selected_text    1
sentiment        0
dtype: int64


In [13]:
data = data.dropna(subset=['text', 'selected_text'])


In [14]:
data.isnull().sum()

textID           0
text             0
selected_text    0
sentiment        0
dtype: int64

In [20]:
import pandas as pd
import re
import string
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk

# Download necessary NLTK data files if not already available
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# 1. Handle missing values - Drop rows with any null values
cleaned_data = data.dropna().reset_index(drop=True)

# 2. Remove duplicates based on 'textID' or 'text'
cleaned_data = cleaned_data.drop_duplicates(subset=['textID', 'text']).reset_index(drop=True)

# 3. Text Cleaning Function
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove special characters and numbers
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\d', '', text)
    # Tokenization and remove stopwords
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatization
    lemmatized = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(lemmatized)

# Apply text cleaning to both 'text' and 'selected_text'
cleaned_data['text'] = cleaned_data['text'].apply(clean_text)
cleaned_data['selected_text'] = cleaned_data['selected_text'].apply(clean_text)

# 4. Sentiment Encoding
sentiment_mapping = {'positive': 2, 'neutral': 1, 'negative': 0}
cleaned_data['sentiment_encoded'] = cleaned_data['sentiment'].map(sentiment_mapping)

# Review the cleaned data
cleaned_data_head = cleaned_data.head()
cleaned_data_info = cleaned_data.info()

cleaned_data_head, cleaned_data_info


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27480 entries, 0 to 27479
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   textID             27480 non-null  object
 1   text               27480 non-null  object
 2   selected_text      27480 non-null  object
 3   sentiment          27480 non-null  object
 4   sentiment_encoded  27480 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.0+ MB


(       textID                            text    selected_text sentiment  \
 0  cb774db0d1                 responded going  responded going   neutral   
 1  549e992a42         sooo sad miss san diego         sooo sad  negative   
 2  088c60f138                    bos bullying         bullying  negative   
 3  9642c003ef           interview leave alone      leave alone  negative   
 4  358bd9e861  son put release already bought              son  negative   
 
    sentiment_encoded  
 0                  1  
 1                  0  
 2                  0  
 3                  0  
 4                  0  ,
 None)

In [23]:
import pandas as pd
import re
import string
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk

# Download necessary NLTK data files if not already available
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# 1. Handle missing values - Drop rows with any null values
cleaned_data = data.dropna().reset_index(drop=True)

# 2. Remove duplicates based on 'textID' or 'text'
cleaned_data = cleaned_data.drop_duplicates(subset=['textID', 'text']).reset_index(drop=True)

# 3. Text Cleaning Function
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove special characters and numbers
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\d', '', text)
    # Tokenization and remove stopwords
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatization
    lemmatized = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(lemmatized)

# Apply text cleaning to both 'text' and 'selected_text'
cleaned_data['text'] = cleaned_data['text'].apply(clean_text)
cleaned_data['selected_text'] = cleaned_data['selected_text'].apply(clean_text)

# 4. Sentiment Encoding
sentiment_mapping = {'positive': 2, 'neutral': 1, 'negative': 0}
cleaned_data['sentiment_encoded'] = cleaned_data['sentiment'].map(sentiment_mapping)

# Review the cleaned data
cleaned_data_head = cleaned_data.head()
cleaned_data_info = cleaned_data.info()

cleaned_data_head, cleaned_data_info


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27480 entries, 0 to 27479
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   textID             27480 non-null  object
 1   text               27480 non-null  object
 2   selected_text      27480 non-null  object
 3   sentiment          27480 non-null  object
 4   sentiment_encoded  27480 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.0+ MB


(       textID                            text    selected_text sentiment  \
 0  cb774db0d1                 responded going  responded going   neutral   
 1  549e992a42         sooo sad miss san diego         sooo sad  negative   
 2  088c60f138                    bos bullying         bullying  negative   
 3  9642c003ef           interview leave alone      leave alone  negative   
 4  358bd9e861  son put release already bought              son  negative   
 
    sentiment_encoded  
 0                  1  
 1                  0  
 2                  0  
 3                  0  
 4                  0  ,
 None)